# Single-task Test-time fine-tuning for ARC24

## Goal

In this notebook I will explore a version of test-time fine-tuning that adapts the base model for each task.

Instead of fine-tuning in all the test tasks together, I will fine-tune a model for each of the tasks. Hopefully this will bring the improvement that Jack Cole is talking about in the MLST podcast.

## Configuration and imports

In [1]:
#qwen25-0.5b/8 100split step32k_bs1 8e-5lr_lin 96E
n_splits = 100 #2, 4, 10, 20, 50, 100
total_train_steps = 32000
class cfg:
    # Model
    model_path = '/kaggle/input/qwen2.5/transformers/0.5b-instruct/1'
    input_lora_path = '/kaggle/input/loras/transformers/qwen2.5-0.5b-instruct/8'
    prompt_version = 'output-from-examples-v1'
    merged_model_path = '/kaggle/tmp/qwen_merged_model'
    grid_encoder = 'GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))' #GridCodeBlockEncoder(MinimalGridEncoder())
    max_model_len = 10240
    # Dataset
    dataset_path = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'
    #dataset_path = '/kaggle/input/arc24-source-code/new_partitions/val_rs7.json'
    #dataset_path = 'smaller_val_challenges.json'
    split_size = 100//n_splits # How many tasks there would be on each split, use 1 for the canonical single-task test-time fine-tuning
    # Fine-tuning params
    max_steps = total_train_steps//n_splits
    learning_rate = 8e-5 #1e-4 for lora smaller than 512
    lr_scheduler_type: str = "linear" #linear, constant_with_warmup, cosine, cosine_with_restarts
    batch_size = 1
    max_seq_len = 5120 #3456, 5120
    # Inference params
    predictions_per_task = 96 # multiple of 8
    inference_timeout = "12m" # max inference time per split, I estimate that for 128 preds Qwen-0.5B takes 6 min in the worst case (50 splits)
    # Ensemble
    ensemble_with_2020: bool = True

import os
is_dry_run = cfg.dataset_path == '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json' and not os.getenv('KAGGLE_IS_COMPETITION_RERUN')
if is_dry_run:
    print('This is a dry run, no inference nor installation of packages will be done')
    
if int(cfg.input_lora_path.split('/')[-1]) < 18 and cfg.input_lora_path.startswith('/kaggle/input/loras/transformers/qwen2-0.5b'):
    assert cfg.prompt_version == 'output-from-examples-v0'
else:
    assert cfg.prompt_version == 'output-from-examples-v1'

This is a dry run, no inference nor installation of packages will be done


In [2]:
import logging
import subprocess
import sys
import json
import glob
import os
import shutil
from tqdm.auto import tqdm

if not is_dry_run:
    sys.path.append('/kaggle/input/arc24-source-code')

# Configure logging to output to the notebook console
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

/opt/anaconda3/envs/arc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0. Launch 2020 solution in the background

In [3]:
if not is_dry_run and cfg.ensemble_with_2020:
    print('Launching 2020 solution in the background')
    args = [
        #'taskset', '-c', '0', apparently this will restrict the job to a single cpu
        'python',
        '/kaggle/input/arc24-source-code/full_2020_solution.py',
        f'--dataset_filepath={cfg.dataset_path}',
        '--icecuber_output_filepath=icecuber_submission.json',
        '--dsl_output_filepath=submission_program_search.json']
    full_2020_solution_process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

## 1. Install libraries and launch resource monitor

In [4]:
%%time
if not is_dry_run:
    !bash /kaggle/input/arc24-source-code/install_libraries.sh

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 3.1 μs


In [5]:
if not is_dry_run:
    from arc24.utils import ResourceMonitor
    monitor = ResourceMonitor(interval=1)
    monitor.start()

## 2. Prepare data for training

In [6]:
if not is_dry_run:
    single_task_datasets_path = 'single_task_datasets'
    os.makedirs(single_task_datasets_path, exist_ok=True)
    with open(cfg.dataset_path, 'r') as f:
        items = list(json.load(f).items())
    
    assert len(items) % cfg.split_size == 0
    print(len(items))
    for batch_idx in tqdm(range(len(items)//cfg.split_size), desc='Creating single task datasets'):
        data = dict(items[batch_idx*cfg.split_size: (batch_idx + 1)*cfg.split_size])
        assert len(data) == cfg.split_size
        task_id = list(data.keys())[0]
        with open(os.path.join(single_task_datasets_path, f'{task_id}.json'), 'w') as f:
            json.dump(data, f)

In [7]:
! ls -lh {single_task_datasets_path}

ls: {single_task_datasets_path}: No such file or directory


In [ ]:
if not is_dry_run:
    training_datasets_path = 'single_task_training_datasets'
    os.makedirs(training_datasets_path, exist_ok=True)
    dataset_filepaths = glob.glob(os.path.join(single_task_datasets_path, '*.json'))
    for dataset_filepath in tqdm(dataset_filepaths, desc='Creating ttft training datasets'):
        !python /kaggle/input/arc24-source-code/create_n-1_dataset.py \
        {dataset_filepath} \
        {os.path.join(training_datasets_path, os.path.basename(dataset_filepath))}

## 3. Test-time fine-tuning

In [ ]:
%%time
def clean_train_output_except_adapter(output_dir):
    """
    Max Disk is 57.6GiB, but around 8GiB are already used. 
    If each checkpoint weights 265M, that would be 26GB for 100 savings. The optimizer was using just 133M.
    However on '/kaggle/working/' we can only save 20GB, the run that had the disk error used 50 splits, each split was using around 400M, so it makes sense
    """
    #!rm -rf {output_dir}/runs
    !rm {output_dir}/*/*.pth {output_dir}/*/*.pt {output_dir}/*/*.md {output_dir}/*/*.txt {output_dir}/*/*.bin {output_dir}/*/token*
    !rm {output_dir}/*/added_tokens.json {output_dir}/*/special_tokens_map.json {output_dir}/*/vocab.json {output_dir}/*/trainer_state.json
    
if not is_dry_run:
    dataset_filepaths = sorted(glob.glob(os.path.join(training_datasets_path, '*.json')))
    checkpoints_folder = '/kaggle/tmp/checkpoints' # https://www.kaggle.com/docs/notebooks#technical-specifications
    os.makedirs(checkpoints_folder, exist_ok=True)
    for dataset_filepath in tqdm(dataset_filepaths, desc='Finetuning models'):
        output_dir = os.path.join(checkpoints_folder, os.path.splitext(os.path.basename(dataset_filepath))[0])
        !python /kaggle/input/arc24-source-code/fine-tuning.py \
        --model_path={cfg.model_path} \
        --adapter_path={cfg.input_lora_path} \
        --output_dir={output_dir} \
        --train_datasets {dataset_filepath} {cfg.prompt_version} \
        --val_dataset {dataset_filepath} {cfg.prompt_version} \
        --max_steps={cfg.max_steps} \
        --eval_steps={cfg.max_steps*2} \
        --max_seq_len={cfg.max_seq_len} \
        --learning_rate={cfg.learning_rate} \
        --lr_scheduler_type={cfg.lr_scheduler_type} \
        --batch_size={cfg.batch_size} \
        --report_to=tensorboard \
        --grid_encoder="{cfg.grid_encoder}" \
        --remove_train_samples_to_fit_max_seq_len \
        --torch_dtype=float16 \
        --no-verbose
        clean_train_output_except_adapter(output_dir)
        logging.info(f'Finished fine-tuning for split {dataset_filepaths.index(dataset_filepath) + 1}/{len(dataset_filepaths)}')

In [ ]:
!ls -lh {checkpoints_folder}/*/checkpoint*/adapter_model.safetensors

## 5. Program search 

https://www.kaggle.com/code/mehrankazeminia/3-arc24-developed-2020-winning-solutions

I'm doing program search before inference because I have observed OOM errors when running icecuber solution and inference in parallel. It seems I can fine-tune and run icecuber solution at the same time without any problem.

In [ ]:
if not is_dry_run and cfg.ensemble_with_2020:
    # old code to call program search dsl sequentially
    #!python /kaggle/input/arc24-source-code/program_search_dsl.py \
    #--dataset_filepath={cfg.dataset_path} \
    #--output_filepath=submission_program_search.json
    
    print('Waiting for icecuber process to end')
    full_2020_solution_process.wait()
    stdout, stderr = full_2020_solution_process.communicate()
    print("Script output:", stdout.decode())
    print("Script errors:", stderr.decode())
    
    #old code to call icecuber solution sequentially
    #!python /kaggle/input/arc24-source-code/icecuber_solution.py \
    #--dataset_filepath={cfg.dataset_path} \
    #--output_filepath=icecuber_submission.json

## 6. Inference

In [ ]:
%%time
if is_dry_run:
    with open('submission.json', 'w') as f:
        json.dump(dict(dry_run=True), f)
else:
    inference_path = 'inference'
    os.makedirs(inference_path)
    os.environ['VLLM_LOGGING_LEVEL'] = 'ERROR'
    dataset_filepaths = sorted(glob.glob(os.path.join(single_task_datasets_path, '*.json')))
    for dataset_filepath in tqdm(dataset_filepaths):
        task_id = os.path.splitext(os.path.basename(dataset_filepath))[0]
        checkpoint_path = os.path.join(checkpoints_folder, task_id, f'checkpoint-{cfg.max_steps}')
        if not os.path.exists(checkpoint_path):
            print(f'Checkpoint path does not exist: {checkpoint_path}')
            checkpoint_path = cfg.input_lora_path
        
        !python /kaggle/input/arc24-source-code/merge_lora.py \
        --base_model_path={cfg.model_path} \
        --lora_path={checkpoint_path} \
        --output_path={cfg.merged_model_path}
        
        output_filepath = os.path.join(inference_path, f'{task_id}_inference.json')
        while not os.path.exists(output_filepath):
            ! timeout {cfg.inference_timeout} python /kaggle/input/arc24-source-code/inference.py\
            --model_path={cfg.merged_model_path} \
            --prompt_version={cfg.prompt_version} \
            --dataset={dataset_filepath} \
            --output_filepath={output_filepath} \
            --max_model_len={cfg.max_model_len} \
            --grid_encoder="{cfg.grid_encoder}" \
            --predictions_per_task={cfg.predictions_per_task}  
            if not os.path.exists(output_filepath):
                print('\t\tWARNING, INFERENCE DID TIMEOUT!')
                
        logging.info(f'Finished inference for split {dataset_filepaths.index(dataset_filepath) + 1}/{len(dataset_filepaths)}')

In [ ]:
# combine all the predictions into single files
if not is_dry_run:
    filepaths = glob.glob(os.path.join(inference_path, '*_inference.json'))
    solutions = dict()
    for filepath in tqdm(filepaths):
        with open(filepath, 'r') as f:
            solutions.update(json.load(f))
    with open('submission_all.json', 'w') as f:
        json.dump(solutions, f)

    filepaths = glob.glob(os.path.join(inference_path, '*_task_results.json'))
    task_results = []
    for filepath in tqdm(filepaths):
        with open(filepath, 'r') as f:
            task_results.extend(json.load(f))
    with open('submission_all_task_results.json', 'w') as f:
        json.dump(task_results, f)

In [ ]:
if not is_dry_run:
    !python /kaggle/input/arc24-source-code/voting.py \
    --input_filepath=submission_all_task_results.json \
    --output_filepath=submission_voting.json  

In [ ]:
if not is_dry_run and cfg.dataset_path != '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json':
    sys.path.append('/kaggle/input/arc24-source-code')
    from evaluation import (
        load_arc_data_with_solutions, evaluate,
        study_effect_of_the_number_of_solutions,
        study_attempt_accuracy,
        visualize_tasks_and_predictions)
    
    print('Results with all the predictions')
    with open('submission_all.json', 'r') as f:
        solutions = json.load(f)
    data = load_arc_data_with_solutions(cfg.dataset_path)
    evaluate(data, solutions)
    
    study_effect_of_the_number_of_solutions(solutions, data)
    visualize_tasks_and_predictions(solutions, data, only_correct=False)
    
    print('Results from selected 2 attemps')
    with open('submission_voting.json', 'r') as f:
        solutions = json.load(f)
    evaluate(data, solutions)
    study_attempt_accuracy(solutions, data)

## 7. Combine solutions

In [ ]:
if not is_dry_run:
    if cfg.ensemble_with_2020:
        !python /kaggle/input/arc24-source-code/combine_submissions.py \
        --sub_1=submission_program_search.json \
        --sub_2=icecuber_submission.json \
        --output=submission_2020.json \
        --give_preference_to_second_submission_on_second_attempt

        !python /kaggle/input/arc24-source-code/combine_submissions.py \
        --sub_1=submission_2020.json \
        --sub_2=submission_voting.json \
        --output=submission.json \
        --give_preference_to_second_submission_on_second_attempt
    else:
        !cp submission_voting.json submission.json

In [ ]:
if not is_dry_run and cfg.dataset_path != '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json':
    print('Results from final submission')
    with open('submission.json', 'r') as f:
        solutions = json.load(f)
    evaluate(data, solutions)
    study_attempt_accuracy(solutions, data)

## 7. Clean

In [ ]:
def clean():
    for filepath in glob.glob('*'):
        if filepath == 'submission.json':
            continue
        if os.path.isdir(filepath):
            shutil.rmtree(filepath)
        else:
            os.remove(filepath)
    !rm -rf /kaggle/tmp/*

clean()
!ls -lh

## 8. Show resources usage

In [ ]:
if not is_dry_run:
    monitor.stop()
    monitor.plot()

## References

- https://www.kaggle.com/code/ironbar/fine-tuned-llms-for-arc24-challenge
- https://www.kaggle.com/code/ironbar/fine-tune-llm-on-arc
- https://www.kaggle.com/code/mehrankazeminia/3-arc24-developed-2020-winning-solutions

## TODO

- [x] Can I log metrics when not using wandb? Yes, simply by setting the log to tensorboard
- [x] Combine with icecuber solution
- [x] Group imports
- [ ] Assert that the solution has all the predictions, if there is a timeout there might be missing predictions. It seems that VLLM can get stuck sometimes.